<a href="https://colab.research.google.com/github/YossefMohamed/Detecting-COVID-19-Through-Sound/blob/main/deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://

In [ ]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!ngrok config add-authtoken 29nINwI8wrDAHlpn7vluNXzocO4_5VXeZmbEtMv4SZybSVaKn

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [ ]:
!ngrok authtoken '29nINwI8wrDAHlpn7vluNXzocO4_5VXeZmbEtMv4SZybSVaKn'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
ls

gdrive/  sample_data/  sample.wav


# PUT SAMPLE.WAV file in path => "/content/sample.wav" before executing

In [ ]:
from flask_ngrok import run_with_ngrok

from flask import Flask , render_template, request

import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import h5py


import numpy as np
import librosa 
import librosa.display
import pylab
import cv2
import os

model  = load_model('/content/gdrive/MyDrive/150/1/145--0.442--0.315.hdf5') # load your model

def feature_extractor(row, x):                 #to extract (MFCC , Mel spectogram) ,take(sound path,name)

  name     = x
  try:
    audio,sr = librosa.load(row)
    #For MFCCS 
    mfccs    = librosa.feature.mfcc(y=audio,sr=22050, n_mfcc=13)
    mfccsscaled = np.mean(mfccs.T,axis=0)
    
    #Mel Spectogram
    pylab.axis('off') # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
    melspec  = librosa.feature.melspectrogram(y=audio,sr=sr)
    s_db     = librosa.power_to_db(melspec, ref=np.max)
    librosa.display.specshow(s_db)

    savepath = os.path.join(name+'.png')
    pylab.savefig(savepath, bbox_inches=None, pad_inches=0)
    pylab.close()
  except:
    print('File cannot open')
    return None,None

  return mfccsscaled,savepath
        
class CustomDataset(tf.keras.utils.Sequence):   # prepar data pefore predction
  def __init__(self,imgfiles,labels,batch_size,target_size=(224,224),shuffle=False,scale=255,n_classes=1,n_channels=3):
    self.batch_size = batch_size
    self.dim        = target_size
    self.labels     = labels
    self.imgfiles   = imgfiles
    self.n_classes  = n_classes
    self.shuffle    = shuffle
    self.n_channels = n_channels
    self.scale      = scale

    self.c          = 0
    self.on_epoch_end()

  def __len__(self):
    # returns the number of batches
    return int(np.floor(len(self.imgfiles) / self.batch_size))

  def __getitem__(self, index):
    # returns one batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Generate data
    X, y = self.__data_generation(indexes)
    return X, y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.imgfiles))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)
  
  def __data_generation(self, list_IDs_temp):
    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
        # Store sample
        img   = cv2.imread(self.imgfiles[ID])
        img   = cv2.resize(img,self.dim,interpolation = cv2.INTER_CUBIC)
        X[i,] = img / self.scale

        # Store class
        y[i] = self.labels[ID]

        self.c +=1
    return X, y #keras.utils.to_categorical(y, num_classes=self.n_classes)


class CustomPipeline(tf.keras.utils.Sequence):
  def __init__(self,data_x,data_y,batch_size=32,shuffle=False,n_classes=1):
    self.features   = data_x
    self.labels     = data_y
    self.batch_size = 1
    self.shuffle    = shuffle
    self.n_features = self.features.shape[1]
    self.n_classes  = 1
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(len(self.features) / self.batch_size))

  def __getitem__(self,index):
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    X, y = self.__data_generation(indexes)
    return X, y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.features))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self,indexes):
    X = np.empty((self.batch_size, self.n_features))
    y = np.empty((self.batch_size), dtype=int)

    for i, ID in enumerate(indexes):
      X[i,] = self.features[ID]
      y[i,] = self.labels[ID]
    return X, y

class TripleInputGenerator(tf.keras.utils.Sequence):
  """Wrapper of two generatos for the combined input model"""

  def __init__(self, X1, X2, X3, Y, batch_size,target_size=(224,224)):
      self.genX1 = CustomPipeline(X1, Y, batch_size=batch_size,shuffle=False)
      self.genX2 = CustomDataset (X2, Y, batch_size=batch_size,shuffle=False,target_size=target_size)
      self.genX3 = CustomPipeline(X3, Y, batch_size=batch_size,shuffle=False)
  def __len__(self):
      return self.genX1.__len__()

  def __getitem__(self, index):
      X1_batch, Y_batch = self.genX1.__getitem__(index)
      X2_batch, Y_batch = self.genX2.__getitem__(index)
      X3_batch, Y_batch = self.genX3.__getitem__(index)

      X_batch = [X1_batch,X1_batch,X2_batch, X3_batch]
      return X_batch, Y_batch

app = Flask(__name__)

run_with_ngrok(app)  
@app.route("/")
def home():
   return {"message" : "hello world"}


@app.route('/sample',methods=['POST'])
def predict():
    # soundfile = request.files['soundfile'] 
    # sound_path = "./sounds/" + soundfile.filename
    # soundfile.save(sound_path)
    # print(request.POST['soundfile'])
    print(request.form["link"])
    # return "Mohaaah"
#######################################
    import requests
    sound_path = request.form["link"] #for test only
    r = requests.get(sound_path)
    print(len(r.content))
    # /content/sample.wav
    with open("/content/sample.wav","wb") as code:
        code.write(r.content)
        code.close()
    sound_path = "/content/sample.wav"
    data_x1 = [] #MFCC
    data_xp1= [] #Mel spectogram
    data_xx1 = [] # external feature
    fever = 0
    breathProblem = 0

    if request.form["fever"] == "false" :
      fever = 0
    else:
      fever = 1
    breathProblem=0
    if request.form["breathProblem"] == "false" :
      breathProblem = 0
    else:
      breathProblem = 1

    

    mfccs,savepath  = feature_extractor(sound_path,"g5f5")  # call feature_extractor
    data_x1.append(mfccs)                                   # append mfcc
    data_xp1.append(savepath)                               # append save path of mel-spectogram 
    data_xx1.append([fever,breathProblem])    # [fever , breathProblem ]                              # enter your external information 
    data_y1 = [1]

    tfeatures =  np.array(data_x1)
    textra    =  np.array(data_xx1)
    timgs     =  data_xp1
    labels    =  np.array(data_y1)


    TEST1 = TripleInputGenerator(tfeatures,timgs,textra,labels,batch_size=1,target_size=(224,224)) # prepare data
    y_score = model.predict(TEST1)                                                                 #predict 


    if(y_score > .5):
          return {"covid":True}
    else:
          return {"covid":False}

##########################################


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://fb2b-34-73-95-240.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Jun/2022 07:50:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 07:50:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 07:50:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 07:50:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Jun/2022 07:51:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 09:06:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 09:06:26] "GET /favicon.ico HTTP/1.1" 404 -
